# 迭代器：你凭什么能for循环迭代列表和元组
* 迭代器允许你创建可被迭代的对象，用于for循环等，比如对一个类迭代

## 定义
迭代器是对可迭代对象的改造升级，一个对象定义了__iter__()方法，那么它就是可迭代对象
如果一个对象同时实现了\__iter__和\__next__方法，那么它就是迭代器。

In [11]:
a=iter([1,2,3])
for i in range(3):
    print(next(a))

1
2
3


## 一般建议\__iter__直接返回其本身，在next中再实现功能
\__next__的作用是返回遍历过程中的下一个元素，如果没有下一个元素则主动抛出StopIteration异常。
* 相当于你可以在next定义每次迭代要做什么，返回什么
* 迭代器会记住迭代到哪步了，比如你在i>10000停止，再次调用并不会从头开始，除非再次实例化

In [14]:
'''一个迭代生成斐波那契数列的类'''
class Fib:
    def __init__(self):
        self.x0=0
        self.x1=1
    def __next__(self):
        '''在next中定义行为'''
        self.x0,self.x1=self.x1,self.x1+self.x0
        return self.x0
    def __iter__(self):
        return self
    
fib=Fib()
for i in fib:
    if i>10000:
        break
    print(i,end=' ')

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 

# 生成器
Python 生成器(Generators)是一种可以迭代对象,可以通过 yield 关键字将函数转化为生成器,本质上就是边循环边处理，实现任务可以暂停，接受信息，再进行，循环往复

生成器的主要特征是:
1. 使用 yield 关键字,而不是 return 语句返回结果
2. 每次调用 next()或进行迭代,会继续执行已经暂停的函数,而不是从头开始执行。
3. 生成器能够记住上一次的执行状态,并在下一次遍历时继续执行。
4. 生成器只在被遍历读取时才会进行计算,不会提前生成全部值放在内存中。这种延迟计算的特性使得生成器可以处理那些不能放入内存的巨大的数据集。
5. 当迭代到最后，开始迭代时后没有发现yield，会抛出异常StopIteration（可以捕捉并处理异常）


以下例子表明生成器一大优点，如果 nested_list 列表包含了数百万个子列表, flatten 函数就会生成一个数百万元素的大列表,占用大量内存。
而使用 yield 的生成器版本不会有这个问题,它只会在遍历时逐个生成元素,不需要创建完整的大列表。

In [1]:
'''例子：降维多维数组（其实用numpy的flatten方法使一样的）'''


def flatten(lst):
    '''生成一个生成器'''
    for sublist in lst:
        for element in sublist:
            ''' yield element 会返回元素 element,并暂停函数执行。'''
            yield element


generator = flatten([[1, 2, 3],
                     [9, 10],
                     [1, 2, 4, 2]])
next(generator)

1

## 递归的生成器
1. 首先尝试将 lst + '',如果成功(lst 是字符串等),则抛出 TypeError。这是因为字符串不能迭代,不能调用 flatten。
2. 如果 lst 不是字符串类对象,则遍历它的每一个子列表 sublist
3. 对每个子列表递归调用 flatten,将其展开
4. 从递归调用中生成的生成器中yield 每个元素,展开子列表
5. 如果在第1步抛出了 TypeError,则直接 yield lst。这是因为如果 lst 是字符串等,不需要展开,直接返回。


In [30]:
def flatten(lst):
    try:
        '''检查是否lst为类似字符串的对象，若没异常（说明类似字符串），就抛出错误'''
        try:
            lst + ''
        except TypeError:
            pass
        else:
            raise TypeError
        for sublist in lst:
            for element in flatten(sublist):
                yield element
    except TypeError:
        yield lst


test = [[[[1, 2, 3], 2, 'saiko'], [9, 2], 1],'zetsubo', 19, [22, 3], 11] #我自己都不知道这嵌套了几层
list(flatten(test))

[1, 2, 3, 2, 'saiko', 9, 2, 1, 'zetsubo', 19, 22, 3, 11]

## send方法
send(value) 方法用于向生成器函数发送值。它允许你在生成器暂停的 yield 语句处注入一个值，成为yield表达式的值，并且自动触发next函数，返回生成器产生的下一个值

**每次调用 generator.send(x) 时:**
1. 生成器会执行到遇到的下一个 yield 语句
1. 将 send 方法传入的值赋值给 yield 语句左边的变量（这时yield表现地像一个表达式，其值就是你send给它的对象）
1. 继续执行直到碰到下一个yield，返回yield右边的对象的值（可以写某种操作，如加减乘除等）
1. 挂起生成器，等待下次迭代
* 仅当生成器被挂起（遇到第一个yield）后使用send才有意义，如果刚开始就想提供信息，可以传递None给它

        如generator.send(None)或者next(generator)
* 生成器相比迭代器允许用户与生成器进行通信，即用户send一个值，生成器yield一个结果给用户

In [52]:
def count_to_ten():
    num = 0
    while num <= 10:
        value = yield num
        print(f'Received {value}')
        num += 1

generator = count_to_ten()
print(generator.send(None))
'''什么也没发，进入到下一个yield 返回num=0的值'''
print(generator.send(1)) 
'''此时发送1给生成器，value被赋值为1,（此时num还是0）
然后num+=1变成1，进入下一个yield，返回num=1'''


0
Received 1
1


### 例子：一个对数据集所有矩阵进行矩阵乘法的例子，但我只要用一点数据就好，一次算太麻烦

解释一下：
1. 用next启动生成器，此时到达第一个yield，None被赋值给son，什么也不返回
2. 传入数据，要计算的矩阵数据被赋值给son，然后计算乘法，yield返回值
3. 继续向前，到达下一个yield（无限循环），挂起生成器等待下一次send

In [72]:
import numpy as np

np.random.seed(10)
dataset = [np.random.randn(2, 2) for i in range(4)]
dataset

[array([[ 1.3315865 ,  0.71527897],
        [-1.54540029, -0.00838385]]),
 array([[ 0.62133597, -0.72008556],
        [ 0.26551159,  0.10854853]]),
 array([[ 0.00429143, -0.17460021],
        [ 0.43302619,  1.20303737]]),
 array([[-0.96506567,  1.02827408],
        [ 0.22863013,  0.44513761]])]

In [76]:
def preprocess():
    parent = np.array([[-1, 0], [0, -1]])
    son = yield
    while True:
        son = yield son @ parent

generator = preprocess()
next(generator)  #启动生成器
print(generator.send(dataset[0]))
print(generator.send(dataset[1]))

[[-1.3315865  -0.71527897]
 [ 1.54540029  0.00838385]]
[[-0.62133597  0.72008556]
 [-0.26551159 -0.10854853]]


In [77]:
generator=preprocess()
next(generator)
for i in dataset:
    print(generator.send(i))

[[-1.3315865  -0.71527897]
 [ 1.54540029  0.00838385]]
[[-0.62133597  0.72008556]
 [-0.26551159 -0.10854853]]
[[-0.00429143  0.17460021]
 [-0.43302619 -1.20303737]]
[[ 0.96506567 -1.02827408]
 [-0.22863013 -0.44513761]]


### 别人的例子

In [51]:
def house_price(areas):
    unit_price = yield # 接收初始的unit_price
    for area in areas:
        unit_price = yield unit_price * area
'''在这段代码中，house_price 是一个生成器函数，
接收一个 areas 列表作为输入参数。在函数内部，
我们使用 yield 语句来定义生成器的行为。
代码的第一行 unit_price = yield 是用于接收初始的 unit_price，
即在调用生成器之后首次发送给生成器的值。
之后，我们使用 for 循环遍历 areas 列表，
每次迭代时都会使用 yield 语句来生成一个值。'''

unit_prices = [1, 2, 3, 1]
areas = [100, 120, 110, 180]
hp = house_price(areas)
hp.send(None) 
'''我们使用 hp.send(None) 来启动生成器，将其执行到第一个 yield 语句处。
此时，生成器会等待接收初始的 unit_price。'''
for unit_price in unit_prices:
    cur_price = hp.send(unit_price)
    print(f"Price: {cur_price}")
    '''接下来的 for 循环用于遍历 unit_prices 列表中的每个元素。
    在每次迭代时，我们使用 hp.send(unit_price)
    将 unit_price 发送给生成器。生成器会接收这个值，
    并执行乘法运算 unit_price * area，然后再次使用 yield 语句生成结果。'''


Price: 100
Price: 240
Price: 330
Price: 180


## 八皇后问题
思想：先在棋盘第一行放第一个皇后，再选择下一行为下一个皇后放位置，当发现无法为一个皇后放位置时，回溯到上一层为上一个皇后改变位置，再到下一步寻找可能位置，如果还不行就回溯到上一个皇后，直到可以。

### 定义判断冲突函数

In [7]:
def conflict(state,nextX):
    '''确定下一个皇后列位置，会不会和前面放好的皇后位置冲突,state为已经有的皇后位置
    state[i]=j表示，第i行的皇后第在j列,注意将行列从0编号'''
    nextY=len(state)
    #nextY为下一个皇后在哪一行（如nextY=2表示在第二行，第一第零行已经摆好）
    for i in range(nextY):
        '''由于我们总在下一行放下一个皇后，所以不用加绝对值和判定两皇后是否在同一行
        只要判断，state[i]-nextX即两皇后的水平距离是否为0（同列）
        或者水平距离会不会等于垂直距离nextY-i（对角线）'''
        if abs(state[i]-nextX) in (0,nextY-i):
            return True
    return False

def queens(num=8,state=()):
    for pos in range(num):
        '''对于每个可能的列位置'''
        if not conflict(state,pos):
            if len(state)==num-1:
                '''如果不冲突且是最后一个皇后，返回其位置的元组
                这个元组会被作为最后一个pos插入到(pos,)+result
                并作为结果返回'''
                yield (pos,)
            else:
                for result in queens(num,state+(pos,)):
                    '''如果不冲突且还没到最后一个皇后，将当前合法的皇后位置插入state'''
                    yield (pos,)+result
len(list(queens(12)))

14200